In [1]:
#!pip install numpy

In [3]:
#!pip install pandas

In [2]:
#!pip install matplotlib

In [4]:
#!pip install imageio

In [5]:
#!pip install netcdf4

In [ ]:
#!pip install pypng

In [1]:
#Manejo de Datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#Imagenes satelitales
from netCDF4 import Dataset, num2date

#Librerias estandar (Extras)
import re
import os
import time
import random
import png
from calendar import monthrange

In [ ]:
"""
DEFINIMOS DIRECTORIOS
"""
path_imagenes = 'D:/GOES/'         #Directorio con las iamgenes satelitales
path_out = 'F:/GOES/'              #Directorio donde se guardara las imagenes procesadas

In [2]:
"""
Definimos los metodos necesarios para 
realizar el procesamiento de las imagenes
"""

'\nDefinimos los metodos necesarios para \nrealizar el procesamiento de las imagenes\n'

In [3]:
#Se obtiene la matriz que representa la imagen satelital con nombre "filename"
def getMatriz(filename, imprimir=0):    
    start_time = time.time()
    try:
        ds = Dataset(filename)
    except:
        print("No se pudo leer el archivo")
        print(filename)
        return np.asarray([-1])
    
    #Obteine los datos de la imagen satelital
    field = ds.variables['CMI'][:].data.astype(np.uint16)
        
    if imprimir:
        print(f"Tiempo en leer imagen satelital: %.5fs" % (time.time() - start_time))
        
    return field

In [4]:
#Devuelve np.Array de las matrices de colores de cada producto en products (C08,C07 o C13)
def leerImagenArea(fecha,path_imagenes, products,times ,imprimir=0): 
    try:   
        year,month,day,hour = fecha.split('-')        
   
    except:
        print("No se pudo leer la fecha")
        print(fecha)
        return [-1]
       
    #Se define el producto 
    mapaArrays = []  
    dimTime = len(times) 
    
    start_time = time.time()
    correcto = True
    for i in range(dimTime):
        productMatriz  =  []
        for product in products:
            hourTemp  = (int(hour)-((i//(dimTime-1)==0)))%24
            if hourTemp == 23:
                filename = f'{path_imagenes}{product}/{year}/{month}/G16_{product}_Cyl_{year}{month}{int(day)-1:02d}-{hourTemp:02d}{times[i]}.nc'            
            else:
                filename = f'{path_imagenes}{product}/{year}/{month}/G16_{product}_Cyl_{year}{month}{day}-{hourTemp:02d}{times[i]}.nc'            
       
            mProduct = getMatriz(filename,imprimir)
            if mProduct.shape == (1,):
                correcto = False
                return np.asarray([-1])
                
            productMatriz.append(mProduct)
        mapaArrays.append(np.dstack(productMatriz))
    
    mapaArrays  = np.array(mapaArrays)  
    if imprimir:
        print(f"Tiempo tomado en obtener matrices de un dato para {len(products)} productos: %.2fs" % (time.time() - start_time))
    
    if not correcto:
        return np.asarray([-1])
    if len(products) == 1:
        return mapaArrays   
    
    return mapaArrays       

In [6]:
#Convierte las imagenes satelitales en un archivo PNG
#Devuelve las fechas con error, que no se logro realizar el archivo PNG
def mergeImagenes(df, path_imagenes, path_out, products, times, imprimir=0,verb=10):    
    dimT = len(times)
    errors = []
    
    # Creamos la carpeta "comprimido"
    try:
        path = os.path.join(path_out, 'PNG')
        os.mkdir(path)
    except FileExistsError:
        pass

    # Auxiliares
    total = len(df) - 1
    completados = 1

    start_time = time.time()
    for i in df.index:
        if not i % verb:
            progreso = completados / total * 100
            f = '{0:.3g}'.format(progreso)
            print(f"Progreso: {(completados * len(products) * dimT)} datos {f}%  en {(time.time() - start_time)}s")
            print(f'Datos con error: {len(errors)}')

        # Creamos la carpeta "month"
        fechaCompleta = df['fecha'][i]
        year, month, day, hour = fechaCompleta.split('-')
        try:            
            pathFecha = os.path.join(path, f'{fechaCompleta}')
            os.mkdir(pathFecha)
        except FileExistsError:
            pass
        except:
            print('No se pudo crear carpeta dia')
       
        matrizGif = leerImagenArea(fechaCompleta,path_imagenes, products,times ,imprimir)
        
        if matrizGif.shape != (1,):            
            try:            
                for j in range(len(times)):
                    outFile = f"{path_out}PNG/{fechaCompleta}/{fechaCompleta}_{j}.png"
                    z = matrizGif[j]                
                    
                    with open(outFile, 'wb') as f:    
                        writer = png.Writer(width=z.shape[1], height=z.shape[0],  bitdepth=16, greyscale=False)  
                        z2list = z.reshape(-1, z.shape[1]*z.shape[2]).tolist()
                        writer.write(f,z2list)                

                completados = completados + 1
            except:
                errors.append(f'Archivo:{fechaCompleta}-{times[j]}')
        else:
            errors.append(f'Archivo:{fechaCompleta}-{times[j]}')      
            print(f'Error: Archivo:{fechaCompleta}-{times[j]}')      
            

    print(f'{len(df)} total de fechas, {len(df) * len(products) * dimT} total de datos procesados')
    print(f'Datos con error {len(errors)}')
    print("Tiempo tomado en comprimir datos: %.2fs" % (time.time() - start_time))
    return errors 

In [28]:
#Comprueba en un dataframe de fechas, si existen las imagenes satelitales para 
#cada canal en products y minuto en timeDim
#Devuelve una lista con lo indices que no se encontraron lso archivos y el producto
def comprobarImagenes(df,products,path_imagenes,timeDim):     
    start_time = time.time()
    no_index = []
    no_product = []
    no_fecha = []
    for i in df.index:       
        year,month,day,hour = df['fecha'][i].split('-')        
        tmpProduct = []        
        for p in products:          
            
            filename = f'{path_imagenes}{p}/{year}/{month}/G16_{p}_Cyl_{year}{month}{day}-{hour}00.nc'                  
            existe = os.path.exists(filename)
            if not existe:
                tmpProduct.append((p,'00'))
                                      
            for td in timeDim[:-1]:
                if int(hour) == 0:
                    filename = f'{path_imagenes}/{p}/{year}/{month}/G16_{p}_Cyl_{year}{month}{int(day)-1:02d}-23{td}.nc'       
                else:
                    filename = f'{path_imagenes}/{p}/{year}/{month}/G16_{p}_Cyl_{year}{month}{day}-{int(hour)-1:02d}{td}.nc'       
                existe = os.path.exists(filename)
                if not existe:
                    tmpProduct.append((p,td))                    
        if len(tmpProduct)>0:
            no_index.append(i)
            no_fecha.append(df['fecha'][i])                
            no_product.append(tmpProduct)
                               
       
    df2 = df.drop(index=no_index)
    print(f'{len(no_index)} datos de precipitaciones eliminados de {len(df)}: No se encontraron los archivos de imagenes satelitales')
    print("Tiempo tomado en verificar datos: %.2fs" % (time.time() - start_time))
    return df2 , (no_fecha,no_product)

In [7]:
"""
Realizmso el procesamiento de las iamgenes
"""

'\nRealizmso el procesamiento de las iamgenes\n'

In [1]:
#Variables generales
products = ['C07','C08','C13']

#!El '00' debe ir al final!
times = ['10','20','30','40','50','00']

dimT = len(times) + 1

In [33]:
#Verificamos si existen las imagenes satelitales para un intervalo de fechas
dfT = pd.DataFrame({'fecha':pd.date_range(start ='12-26-2020',end ='01-01-2021', freq ='H')})
dfT['fecha'] = dfT['fecha'].dt.strftime('%Y-%m-%d-%H')       
dfVerificado, (no_i,no_p) = comprobarImagenes(dfT,products,path_imagenes,times)

123 datos de precipitaciones eliminados de 8785: No se encontraron los archivos de imagenes satelitales
Tiempo tomado en verificar datos: 47.85s


In [34]:
#Se realiza la transformacion a PNG para la fecha verificada
errors = mergeImagenes(dfVerificado,path_imagenes,path_out,products,times,0,50)

No se pudo leer el archivo
D:/GOES/C07/2020/01/G16_C07_Cyl_20200100-2300.nc
Error: Archivo:2020-01-01-23-00
Progreso: 882 datos 0.566%  en 168.4178431034088s
Datos con error: 1
Progreso: 1782 datos 1.14%  en 363.99814224243164s
Datos con error: 1
Progreso: 2610 datos 1.67%  en 524.1096181869507s
Datos con error: 1
No se pudo leer el archivo
D:/GOES/C07/2020/01/G16_C07_Cyl_20200106-2300.nc
Error: Archivo:2020-01-07-23-00
Progreso: 4284 datos 2.75%  en 839.0966773033142s
Datos con error: 2
Progreso: 5184 datos 3.33%  en 997.5041382312775s
Datos con error: 2
Progreso: 6084 datos 3.9%  en 1155.3879673480988s
Datos con error: 2
Progreso: 6984 datos 4.48%  en 1307.7425334453583s
Datos con error: 2
Progreso: 7884 datos 5.06%  en 1462.9348981380463s
Datos con error: 2
Progreso: 8784 datos 5.63%  en 1612.4889187812805s
Datos con error: 2
Progreso: 9684 datos 6.21%  en 1764.3134739398956s
Datos con error: 2
Progreso: 10548 datos 6.77%  en 1906.793446302414s
Datos con error: 2
Progreso: 11448 dat

No se pudo leer el archivo
D:/GOES/C07/2020/08/G16_C07_Cyl_20200800-2300.nc
Error: Archivo:2020-08-01-23-00
Progreso: 91170 datos 58.5%  en 16518.616871356964s
Datos con error: 12
Progreso: 92070 datos 59.1%  en 16702.234305620193s
Datos con error: 12
Progreso: 92934 datos 59.6%  en 16875.1019282341s
Datos con error: 12
Progreso: 93834 datos 60.2%  en 17052.41335749626s
Datos con error: 12
Progreso: 94734 datos 60.8%  en 17229.4298081398s
Datos con error: 12
Progreso: 95544 datos 61.3%  en 17390.079261779785s
Datos con error: 12
Progreso: 96426 datos 61.9%  en 17565.821244955063s
Datos con error: 12
Progreso: 97326 datos 62.4%  en 17744.61187696457s
Datos con error: 12
Progreso: 98226 datos 63%  en 17923.508246660233s
Datos con error: 12
Progreso: 99126 datos 63.6%  en 18097.86159968376s
Datos con error: 12
Progreso: 100026 datos 64.2%  en 18268.12455511093s
Datos con error: 12
Progreso: 100926 datos 64.7%  en 18447.373161315918s
Datos con error: 12
Progreso: 101808 datos 65.3%  en 186